Music Generation System

Import essential libraries for data processing, model building, and MIDI file handling.

numpy: For numerical computations.

tensorflow/keras: For deep learning model creation.

music21: For parsing and processing MIDI files.

os: To work with file paths.

tqdm: To display progress bars.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from music21 import note, chord, stream, converter, instrument
import os
from tqdm import tqdm

Encapsulates all functionalities for training, generating, and saving a music model.


prepare_sequences: Converts notes to numerical sequences for model input/output.

create_model: Builds an LSTM-based model for music generation.

train: Prepares data from MIDI files and trains the model.

save_model: Saves the trained model to a file.

generate_music: Produces a sequence of notes from a starting pattern.

create_midi: Converts the generated notes back into a MIDI file.

In [3]:
class MusicGenerator:
    def __init__(self, sequence_length=100):
        self.sequence_length = sequence_length
        self.notes = []
        self.note_to_int = {}
        self.int_to_note = {}
        self.model = None

    def prepare_sequences(self, notes):
        unique_notes = sorted(set(notes))
        self.note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
        self.int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

        network_input = []
        network_output = []

        for i in range(0, len(notes) - self.sequence_length, 1):
            sequence_in = notes[i:i + self.sequence_length]
            sequence_out = notes[i + self.sequence_length]
            network_input.append([self.note_to_int[char] for char in sequence_in])
            network_output.append(self.note_to_int[sequence_out])

        n_patterns = len(network_input)
        n_vocab = len(unique_notes)

        network_input = np.reshape(network_input, (n_patterns, self.sequence_length, 1))
        network_input = network_input / float(n_vocab)

        network_output = tf.keras.utils.to_categorical(network_output)

        return network_input, network_output, n_vocab

    def create_model(self, n_vocab):
        model = models.Sequential()

        model.add(layers.LSTM(256, input_shape=(self.sequence_length, 1), return_sequences=True))
        model.add(layers.Dropout(0.3))

        model.add(layers.LSTM(512, return_sequences=True))
        model.add(layers.Dropout(0.3))

        model.add(layers.LSTM(256))
        model.add(layers.Dropout(0.3))

        model.add(layers.Dense(256))
        model.add(layers.Dropout(0.3))

        model.add(layers.Dense(n_vocab, activation='softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        self.model = model
        return model

    def train(self, midi_files_path, epochs=50, batch_size=64):
        for file in os.listdir(midi_files_path):
            if file.endswith(('.mid', '.midi')):
                try:
                    midi_path = os.path.join(midi_files_path, file)
                    midi = converter.parse(midi_path)
                    notes_to_parse = None

                    try:
                        s2 = instrument.partitionByInstrument(midi)
                        notes_to_parse = s2.parts[0].recurse()
                    except:
                        notes_to_parse = midi.flat.notes

                    for element in notes_to_parse:
                        if isinstance(element, note.Note):
                            self.notes.append(str(element.pitch))
                        elif isinstance(element, chord.Chord):
                            self.notes.append('.'.join(str(n) for n in element.normalOrder))

                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")
                    continue

        if not self.notes:
            raise ValueError("No valid MIDI files found or no notes extracted")

        # Prepare sequences
        network_input, network_output, n_vocab = self.prepare_sequences(self.notes)

        # Create and train model
        model = self.create_model(n_vocab)
        model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size)

    def save_model(self):
        self.model.save('model.h5')
        print('Model saved to disk')

    def generate_music(self, start_sequence, length=500):
        pattern = start_sequence
        prediction_output = []

        for _ in tqdm(range(length), desc="Generating notes"):
            prediction_input = np.reshape(pattern, (1, len(pattern), 1))
            prediction_input = prediction_input / float(len(self.note_to_int))

            prediction = self.model.predict(prediction_input, verbose=0)
            idx = np.argmax(prediction)
            result = self.int_to_note[idx]
            prediction_output.append(result)

            pattern = np.append(pattern[1:], idx)

        return prediction_output

    def create_midi(self, prediction_output, filename="generated_music.mid"):
        print("Creating MIDI file...")
        offset = 0
        output_notes = []

        for pattern in tqdm(prediction_output, desc="Converting to MIDI"):
            if ('.' in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split('.')
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            else:
                new_note = note.Note(pattern)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)

            offset += 0.5

        midi_stream = stream.Stream(output_notes)
        midi_stream.write('midi', fp=filename)

Creates an instance of the MusicGenerator class.

In [4]:
generator = MusicGenerator()

Trains the model using MIDI files stored in the midi_datasets directory. The training involves:
Parsing MIDI files to extract notes/chords.
Preparing sequences for the LSTM model.
Training the LSTM-based model for note prediction.

Low accuracy doesn't necessarily mean bad music - there are many valid next notes in a sequence.

The model might generate pleasant music even with seemingly low accuracy scores

In [5]:
generator.train("midi_datasets")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 72s 44ms/step - accuracy: 0.0163 - loss: 5.4115
Epoch 2/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 84s 46ms/step - accuracy: 0.0198 - loss: 5.2512
Epoch 3/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.0200 - loss: 5.2359
Epoch 4/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.0219 - loss: 5.2238
Epoch 5/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.0260 - loss: 5.1943
Epoch 6/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.0282 - loss: 5.1670
Epoch 7/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.0308 - loss: 5.1443
Epoch 8/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.0359 - loss: 5.1085
Epoch 9/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 83s 46ms/step - accuracy: 0.0411 - loss: 5.0662
Epoch 10/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.0439 - loss: 5.0277
Epoch 11/50
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.0494 - loss: 4.9743
Epoch 12

Saves the trained model to a file (model.h5) for later use.

In [6]:
generator.save_model()

Model saved to disk


Generates new music notes based on a starting sequence.

start_sequence: Converts the first 100 notes of the dataset into numerical form.

generate_music: Predicts the next sequence of notes based on the trained model.

In [7]:
start_sequence = [generator.note_to_int[note] for note in generator.notes[:100]]
generated_notes = generator.generate_music(start_sequence)

Generating notes: 100%|██████████| 500/500 [00:34<00:00, 14.56it/s]


Converts the generated notes into a MIDI file (generated_music.mid).

In [8]:
generator.create_midi(generated_notes)

Creating MIDI file...


Converting to MIDI: 100%|██████████| 500/500 [00:00<00:00, 20499.82it/s]
